Manual: https://arxiv.org/help/api/user-manual#Quickstart

In [155]:
import urllib, time, pandas as pd, numpy as np
from bs4 import BeautifulSoup
from enum import Enum

Get metadata from arXiv API:

In [133]:
# Specify categories to search
categories = [
    'astro-ph',     # general astrophysics
    'astro-ph.ga',  # astrophysics of galaxies
    'astro-ph.co',  # cosmology and nongalactic astrophysics
    'astro-ph.ep',  # earth and planetary astrophysics
    'astro-ph.he',  # high energy astrophysical phenomena
    'astro-ph.im',  # instrumentation and methods for astrophysics 
    'astro-ph.sr']  # solar and stellar astrophysics

# Base API query URL
base_url = 'http://export.arxiv.org/api/query?'
results_per_page = 10

rows = []

for category in categories:
    # Search parameters
    search_query = 'cat:' + category
    startIndex = 0
    
    # Loop until total results reached
    while True:
        if startIndex == 0:
            total_results = int(soup.find('opensearch:totalResults').string)
        
        url = 'http://export.arxiv.org/api/query?search_query=' + search_query + '&start=' + str(startIndex)
        results = urllib.request.urlopen(url).read()

        soup = BeautifulSoup(results, 'xml')
        # Get all entry tags
        entries = soup.find_all('entry')

        for entry in entries: 
            # Collect authors
            authors = []
            for author in entry.find_all('name'):
                authors.append(author.string)
            author_str = ', '.join(authors)

            # Get DOI if it exists
            doi = entry.find('arxiv:doi')
            if doi:
                doi = doi.string

            # Extract links if they exist
            doi_link = None
            pdf_link = None
            links = entry.find_all('link')
            for link in links:
                link_title = link.get('title')
                if link_title and link_title == 'doi':
                    doi_link = link['href']
                elif link_title and link_title == 'pdf':
                    pdf_link = link['href']

            # Get journal if it exists
            journal = entry.find('arxiv:journal_ref')
            if journal:
                journal = journal.string

            # Get comment if it exists
            comment = entry.find('arxiv:comment')
            if comment:
                comment = comment.string

            row = {
                'id': entry.id.string,
                'updated': entry.updated.string,
                'published': entry.published.string,
                'title': entry.title.string,
                'summary': entry.summary.string,
                'authors': author_str,
                'doi': doi,
                'doi_link': doi_link,
                'journal': journal,
                'pdf_link': pdf_link,
                'category': entry.find('arxiv:primary_category')['term'],
                'comment': comment
            }
            rows.append(row)
            
            # Quit looping if we have gotten all results
            # print(str(len(rows)) + ' of ' + str(total_results) + '...')
        if len(rows) == total_results:
            break
        else: 
            startIndex += results_per_page
            #time.sleep(3) # recommended to sleep

1 of 105380...
2 of 105380...
3 of 105380...
4 of 105380...
5 of 105380...
6 of 105380...
7 of 105380...
8 of 105380...
9 of 105380...
10 of 105380...
11 of 105380...
12 of 105380...
13 of 105380...
14 of 105380...
15 of 105380...
16 of 105380...
17 of 105380...
18 of 105380...
19 of 105380...
20 of 105380...
21 of 105380...
22 of 105380...
23 of 105380...
24 of 105380...
25 of 105380...
26 of 105380...
27 of 105380...
28 of 105380...
29 of 105380...
30 of 105380...
31 of 105380...
32 of 105380...
33 of 105380...
34 of 105380...
35 of 105380...
36 of 105380...
37 of 105380...
38 of 105380...
39 of 105380...
40 of 105380...
41 of 105380...
42 of 105380...
43 of 105380...
44 of 105380...
45 of 105380...
46 of 105380...
47 of 105380...
48 of 105380...
49 of 105380...
50 of 105380...
51 of 105380...
52 of 105380...
53 of 105380...
54 of 105380...
55 of 105380...
56 of 105380...
57 of 105380...
58 of 105380...
59 of 105380...
60 of 105380...
61 of 105380...
62 of 105380...
63 of 105380...
6

491 of 105380...
492 of 105380...
493 of 105380...
494 of 105380...
495 of 105380...
496 of 105380...
497 of 105380...
498 of 105380...
499 of 105380...
500 of 105380...
501 of 105380...
502 of 105380...
503 of 105380...
504 of 105380...
505 of 105380...
506 of 105380...
507 of 105380...
508 of 105380...
509 of 105380...
510 of 105380...
511 of 105380...
512 of 105380...
513 of 105380...
514 of 105380...
515 of 105380...
516 of 105380...
517 of 105380...
518 of 105380...
519 of 105380...
520 of 105380...
521 of 105380...
522 of 105380...
523 of 105380...
524 of 105380...
525 of 105380...
526 of 105380...
527 of 105380...
528 of 105380...
529 of 105380...
530 of 105380...
531 of 105380...
532 of 105380...
533 of 105380...
534 of 105380...
535 of 105380...
536 of 105380...
537 of 105380...
538 of 105380...
539 of 105380...
540 of 105380...
541 of 105380...
542 of 105380...
543 of 105380...
544 of 105380...
545 of 105380...
546 of 105380...
547 of 105380...
548 of 105380...
549 of 105380.

981 of 105380...
982 of 105380...
983 of 105380...
984 of 105380...
985 of 105380...
986 of 105380...
987 of 105380...
988 of 105380...
989 of 105380...
990 of 105380...
991 of 105380...
992 of 105380...
993 of 105380...
994 of 105380...
995 of 105380...
996 of 105380...
997 of 105380...
998 of 105380...
999 of 105380...
1000 of 105380...
1001 of 105380...
1002 of 105380...
1003 of 105380...
1004 of 105380...
1005 of 105380...
1006 of 105380...
1007 of 105380...
1008 of 105380...
1009 of 105380...
1010 of 105380...
1011 of 105380...
1012 of 105380...
1013 of 105380...
1014 of 105380...
1015 of 105380...
1016 of 105380...
1017 of 105380...
1018 of 105380...
1019 of 105380...
1020 of 105380...
1021 of 105380...
1022 of 105380...
1023 of 105380...
1024 of 105380...
1025 of 105380...
1026 of 105380...
1027 of 105380...
1028 of 105380...
1029 of 105380...
1030 of 105380...
1031 of 105380...
1032 of 105380...
1033 of 105380...
1034 of 105380...
1035 of 105380...
1036 of 105380...
1037 of 105

1441 of 105380...
1442 of 105380...
1443 of 105380...
1444 of 105380...
1445 of 105380...
1446 of 105380...
1447 of 105380...
1448 of 105380...
1449 of 105380...
1450 of 105380...
1451 of 105380...
1452 of 105380...
1453 of 105380...
1454 of 105380...
1455 of 105380...
1456 of 105380...
1457 of 105380...
1458 of 105380...
1459 of 105380...
1460 of 105380...
1461 of 105380...
1462 of 105380...
1463 of 105380...
1464 of 105380...
1465 of 105380...
1466 of 105380...
1467 of 105380...
1468 of 105380...
1469 of 105380...
1470 of 105380...
1471 of 105380...
1472 of 105380...
1473 of 105380...
1474 of 105380...
1475 of 105380...
1476 of 105380...
1477 of 105380...
1478 of 105380...
1479 of 105380...
1480 of 105380...
1481 of 105380...
1482 of 105380...
1483 of 105380...
1484 of 105380...
1485 of 105380...
1486 of 105380...
1487 of 105380...
1488 of 105380...
1489 of 105380...
1490 of 105380...
1491 of 105380...
1492 of 105380...
1493 of 105380...
1494 of 105380...
1495 of 105380...
1496 of 10

1901 of 105380...
1902 of 105380...
1903 of 105380...
1904 of 105380...
1905 of 105380...
1906 of 105380...
1907 of 105380...
1908 of 105380...
1909 of 105380...
1910 of 105380...
1911 of 105380...
1912 of 105380...
1913 of 105380...
1914 of 105380...
1915 of 105380...
1916 of 105380...
1917 of 105380...
1918 of 105380...
1919 of 105380...
1920 of 105380...
1921 of 105380...
1922 of 105380...
1923 of 105380...
1924 of 105380...
1925 of 105380...
1926 of 105380...
1927 of 105380...
1928 of 105380...
1929 of 105380...
1930 of 105380...
1931 of 105380...
1932 of 105380...
1933 of 105380...
1934 of 105380...
1935 of 105380...
1936 of 105380...
1937 of 105380...
1938 of 105380...
1939 of 105380...
1940 of 105380...
1941 of 105380...
1942 of 105380...
1943 of 105380...
1944 of 105380...
1945 of 105380...
1946 of 105380...
1947 of 105380...
1948 of 105380...
1949 of 105380...
1950 of 105380...
1951 of 105380...
1952 of 105380...
1953 of 105380...
1954 of 105380...
1955 of 105380...
1956 of 10

...

10611 of 105380...
10612 of 105380...
10613 of 105380...
10614 of 105380...
10615 of 105380...
10616 of 105380...
10617 of 105380...
10618 of 105380...
10619 of 105380...
10620 of 105380...
10621 of 105380...
10622 of 105380...
10623 of 105380...
10624 of 105380...
10625 of 105380...
10626 of 105380...
10627 of 105380...
10628 of 105380...
10629 of 105380...
10630 of 105380...
10631 of 105380...
10632 of 105380...
10633 of 105380...
10634 of 105380...
10635 of 105380...
10636 of 105380...
10637 of 105380...
10638 of 105380...
10639 of 105380...
10640 of 105380...
10641 of 105380...
10642 of 105380...
10643 of 105380...
10644 of 105380...
10645 of 105380...
10646 of 105380...
10647 of 105380...
10648 of 105380...
10649 of 105380...
10650 of 105380...
10651 of 105380...
10652 of 105380...
10653 of 105380...
10654 of 105380...
10655 of 105380...
10656 of 105380...
10657 of 105380...
10658 of 105380...
10659 of 105380...
10660 of 105380...
10661 of 105380...
10662 of 105380...
10663 of 105

11051 of 105380...
11052 of 105380...
11053 of 105380...
11054 of 105380...
11055 of 105380...
11056 of 105380...
11057 of 105380...
11058 of 105380...
11059 of 105380...
11060 of 105380...
11061 of 105380...
11062 of 105380...
11063 of 105380...
11064 of 105380...
11065 of 105380...
11066 of 105380...
11067 of 105380...
11068 of 105380...
11069 of 105380...
11070 of 105380...
11071 of 105380...
11072 of 105380...
11073 of 105380...
11074 of 105380...
11075 of 105380...
11076 of 105380...
11077 of 105380...
11078 of 105380...
11079 of 105380...
11080 of 105380...
11081 of 105380...
11082 of 105380...
11083 of 105380...
11084 of 105380...
11085 of 105380...
11086 of 105380...
11087 of 105380...
11088 of 105380...
11089 of 105380...
11090 of 105380...
11091 of 105380...
11092 of 105380...
11093 of 105380...
11094 of 105380...
11095 of 105380...
11096 of 105380...
11097 of 105380...
11098 of 105380...
11099 of 105380...
11100 of 105380...
11101 of 105380...
11102 of 105380...
11103 of 105

11491 of 105380...
11492 of 105380...
11493 of 105380...
11494 of 105380...
11495 of 105380...
11496 of 105380...
11497 of 105380...
11498 of 105380...
11499 of 105380...
11500 of 105380...
11501 of 105380...
11502 of 105380...
11503 of 105380...
11504 of 105380...
11505 of 105380...
11506 of 105380...
11507 of 105380...
11508 of 105380...
11509 of 105380...
11510 of 105380...
11511 of 105380...
11512 of 105380...
11513 of 105380...
11514 of 105380...
11515 of 105380...
11516 of 105380...
11517 of 105380...
11518 of 105380...
11519 of 105380...
11520 of 105380...
11521 of 105380...
11522 of 105380...
11523 of 105380...
11524 of 105380...
11525 of 105380...
11526 of 105380...
11527 of 105380...
11528 of 105380...
11529 of 105380...
11530 of 105380...
11531 of 105380...
11532 of 105380...
11533 of 105380...
11534 of 105380...
11535 of 105380...
11536 of 105380...
11537 of 105380...
11538 of 105380...
11539 of 105380...
11540 of 105380...
11541 of 105380...
11542 of 105380...
11543 of 105

11931 of 105380...
11932 of 105380...
11933 of 105380...
11934 of 105380...
11935 of 105380...
11936 of 105380...
11937 of 105380...
11938 of 105380...
11939 of 105380...
11940 of 105380...
11941 of 105380...
11942 of 105380...
11943 of 105380...
11944 of 105380...
11945 of 105380...
11946 of 105380...
11947 of 105380...
11948 of 105380...
11949 of 105380...
11950 of 105380...
11951 of 105380...
11952 of 105380...
11953 of 105380...
11954 of 105380...
11955 of 105380...
11956 of 105380...
11957 of 105380...
11958 of 105380...
11959 of 105380...
11960 of 105380...
11961 of 105380...
11962 of 105380...
11963 of 105380...
11964 of 105380...
11965 of 105380...
11966 of 105380...
11967 of 105380...
11968 of 105380...
11969 of 105380...
11970 of 105380...
11971 of 105380...
11972 of 105380...
11973 of 105380...
11974 of 105380...
11975 of 105380...
11976 of 105380...
11977 of 105380...
11978 of 105380...
11979 of 105380...
11980 of 105380...
11981 of 105380...
11982 of 105380...
11983 of 105

KeyboardInterrupt: 

In [131]:
arxiv_metadata_df = pd.DataFrame(rows)
arxiv_metadata_df

,authors,category,comment,doi,doi_link,id,journal,pdf_link,published,summary,title,updated
0,"Ramesh Narayan, Bohdan Paczyński, Tsvi Piran",astro-ph,14 pages,10.1086/186493,http://dx.doi.org/10.1086/186493,http://arxiv.org/abs/astro-ph/9204001v1,Astrophys.J. 395 (1992) L83-L86,http://arxiv.org/pdf/astro-ph/9204001v1,1992-04-13T18:20:01Z,It is proposed that gamma-ray bursts are cre...,Gamma-Ray Bursts as the Death Throes of Massiv...,1992-04-13T18:20:01Z
1,"Lawrence Krauss, Martin White",astro-ph,13 pages plus figures (not included),10.1086/171792,http://dx.doi.org/10.1086/171792,http://arxiv.org/abs/astro-ph/9204002v1,"Astrophys.J.397:357,1992",http://arxiv.org/pdf/astro-ph/9204002v1,1992-04-26T17:54:00Z,The four observables associated with gravita...,Gravitational Lensing and the Variability of G,1992-04-26T17:54:00Z
2,J. I. Katz,astro-ph,10 pages (Replaced to provide omitted line.),10.1007/BF00645080,http://dx.doi.org/10.1007/BF00645080,http://arxiv.org/abs/astro-ph/9204003v2,None,http://arxiv.org/pdf/astro-ph/9204003v2,1992-04-29T16:36:30Z,The BATSE experiment on GRO has demonstrated...,The Ptolemaic Gamma-Ray Burst Universe,1992-04-30T20:39:38Z
3,"B P Schmidt, R P Kirshner, R G Eastman",astro-ph,21 pages,10.1086/171659,http://dx.doi.org/10.1086/171659,http://arxiv.org/abs/astro-ph/9204004v1,Astrophys.J. 395 (1992) 366-386,http://arxiv.org/pdf/astro-ph/9204004v1,1992-04-30T19:20:04Z,We use the Expanding Photosphere Method to d...,Expanding Photospheres of Type II Supernovae a...,1992-04-30T19:20:04Z
4,"B. J. Carrigan, J. I. Katz",astro-ph,24 pages,10.1086/171906,http://dx.doi.org/10.1086/171906,http://arxiv.org/abs/astro-ph/9204005v1,Astrophys.J. 399 (1992) 100-107,http://arxiv.org/pdf/astro-ph/9204005v1,1992-04-30T19:18:05Z,We have calculated gamma-ray radiative trans...,Radiation Transfer in Gamma-Ray Bursts,1992-04-30T19:18:05Z
5,"D. J. Johnson, M. W. Friedlander, J. I. Katz",astro-ph,29 pages,10.1086/172552,http://dx.doi.org/10.1086/172552,http://arxiv.org/abs/astro-ph/9204006v1,None,http://arxiv.org/pdf/astro-ph/9204006v1,1992-04-30T19:30:14Z,Dust is observed to form in nova ejecta. The...,Nova Dust Nucleation: Kinetics and Photodissoc...,1992-04-30T19:30:14Z
6,Valerio Faraoni,astro-ph,12 pages,10.1086/171866,http://dx.doi.org/10.1086/171866,http://arxiv.org/abs/astro-ph/9205001v1,"Astrophys.J.398:425,1992",http://arxiv.org/pdf/astro-ph/9205001v1,1992-05-01T16:41:45Z,We apply Perlick's (1990a) rigorous formulat...,Nonstationary Gravitational Lenses and the Fer...,1992-05-01T16:41:45Z
7,"T. Hanawa, R. Matsumoto, K. Shibata",astro-ph,12 pages,10.1086/186454,http://dx.doi.org/10.1086/186454,http://arxiv.org/abs/astro-ph/9205002v1,Astrophys.J. 393 (1992) L71-L74,http://arxiv.org/pdf/astro-ph/9205002v1,1992-05-02T02:12:56Z,The effect of the magnetic skew on the Parke...,Giant Molecular Cloud Formation through the Pa...,1992-05-02T02:12:56Z
8,J. I. Katz,astro-ph,3 pages,10.1063/1.42698,http://dx.doi.org/10.1063/1.42698,http://arxiv.org/abs/astro-ph/9205003v1,None,http://arxiv.org/pdf/astro-ph/9205003v1,1992-05-04T18:57:07Z,I present a model for acceleration of proton...,Particle Acceleration in (by) Accretion Discs,1992-05-04T18:57:07Z
9,"A. Cappi, S. Maurogordato",astro-ph,20 pages,None,None,http://arxiv.org/abs/astro-ph/9205004v1,"Astron.Astrophys.259:423-434,1992",http://arxiv.org/pdf/astro-ph/9205004v1,1992-05-08T12:22:00Z,We compare the spatial distributions of gala...,The Spatial Distribution of Nearby Galaxy Clus...,1992-05-08T12:22:00Z


## OAI-PMH Harvesting:

In [168]:
url = 'http://export.arxiv.org/oai2?verb=ListSets'
results = urllib.request.urlopen(url).read()
soup = BeautifulSoup(results, 'xml')
sets = soup.find_all('setSpec')
for s in sets:
    print(s.string)

cs
econ
eess
math
physics
physics:astro-ph
physics:cond-mat
physics:gr-qc
physics:hep-ex
physics:hep-lat
physics:hep-ph
physics:hep-th
physics:math-ph
physics:nlin
physics:nucl-ex
physics:nucl-th
physics:physics
physics:quant-ph
q-bio
q-fin
stat


We are interested in the `physics:astro-ph` set. 

In [191]:
url = 'http://export.arxiv.org/oai2?verb=ListIdentifiers&set=physics:astro-ph&metadataPrefix=arXiv'
results = urllib.request.urlopen(url).read()
soup = BeautifulSoup(results, 'xml')
resumptionToken = soup.find('resumptionToken')
print('Number of article identifiers obtained: ' + str(len(soup.find_all('identifier'))))
print('Total number of articles: ' + str(resumptionToken['completeListSize']))
print('Resumption token: ' + resumptionToken.string)
print('Date that we made this request: ' + soup.find('responseDate').string)
print()

print(soup.prettify())

Number of article identifiers obtained: 10000
Total number of articles: 250047
Resumption token: 3350469|10001
Date that we made this request: 2019-02-08T18:47:40Z

<?xml version="1.0" encoding="utf-8"?>
<OAI-PMH xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd">
 <responseDate>
  2019-02-08T18:47:40Z
 </responseDate>
 <request metadataPrefix="arXiv" set="physics:astro-ph" verb="ListIdentifiers">
  http://export.arxiv.org/oai2
 </request>
 <ListIdentifiers>
  <header>
   <identifier>
    oai:arXiv.org:0704.0009
   </identifier>
   <datestamp>
    2010-03-18
   </datestamp>
   <setSpec>
    physics:astro-ph
   </setSpec>
  </header>
  <header>
   <identifier>
    oai:arXiv.org:0704.0017
   </identifier>
   <datestamp>
    2009-06-23
   </datestamp>
   <setSpec>
    physics:astro-ph
   </setSpec>
  </header>
  <header>
   <identifier>


As it returns only 10k identifiers, we need to get the rest through a batch approach using `resumptionToken`. We also need to pause ~20s, otherwise we get a status `503` error.

https://academia.stackexchange.com/questions/38969/getting-a-dump-of-arxiv-metadata
http://www.openarchives.org/OAI/2.0/openarchivesprotocol.htm#HTTPResponseFormat

In [197]:
url = 'http://export.arxiv.org/oai2?verb=ListIdentifiers&set=physics:astro-ph&metadataPrefix=arXiv'
datestamps = np.array([])
identifiers = np.array([])
setSpecs = np.array([])
resumptionToken = 'placeholder'
responseDate = None

while resumptionToken is not None:
    # Send query and receive results
    results = urllib.request.urlopen(url).read()
    # Parse with Beautiful Soup
    soup = BeautifulSoup(results, 'xml')
    datestamps = np.concatenate([datestamps, [date.text for date in soup.find_all('datestamp')]])
    identifiers = np.concatenate([identifiers, [identifier.text for identifier in soup.find_all('identifier')]])
    setSpecs = np.concatenate([setSpecs, [setSpec.text for setSpec in soup.find_all('setSpec')]])
    resumptionToken = soup.find('resumptionToken')
    if resumptionToken is not None:
        print(str(len(identifiers)) +'/' + str(resumptionToken['completeListSize']) + '...')
        resumptionToken = resumptionToken.text
        url = 'http://export.arxiv.org/oai2?verb=ListIdentifiers&resumptionToken=' + resumptionToken
        time.sleep(20) # avoid http 503 status
    else:
        # Obtain date of last request
        responseDate = soup.find('responseDate').string

10000/250047...
20000/250047...
30000/250047...
40000/250047...
50000/250047...
60000/250047...
70000/250047...
80000/250047...
90000/250047...
100000/250047...
110000/250047...
120000/250047...
130000/250047...
140000/250047...
150000/250047...
160000/250047...
170000/250047...
180000/250047...
190000/250047...
200000/250047...
210000/250047...
220000/250047...
230000/250047...
240000/250047...
250000/250047...
250047/250047...


Load a data frame with metadata for each astro-ph article: 

In [214]:
identifiers_df = pd.DataFrame({
                            'identifier': identifiers,
                            'date_uploaded': datestamps,
                            'setSpec': setSpecs,
                            'date_retrieved': np.full(len(identifiers), responseDate)})

,identifier,date_uploaded,setSpec,date_retrieved
0,oai:arXiv.org:0704.0009,2010-03-18,physics:astro-ph,2019-02-08T19:12:29Z
1,oai:arXiv.org:0704.0017,2009-06-23,physics:astro-ph,2019-02-08T19:12:29Z
2,oai:arXiv.org:0704.0023,2009-06-23,physics:astro-ph,2019-02-08T19:12:29Z
3,oai:arXiv.org:0704.0044,2015-05-13,physics:astro-ph,2019-02-08T19:12:29Z
4,oai:arXiv.org:0704.0048,2008-11-26,physics:astro-ph,2019-02-08T19:12:29Z
5,oai:arXiv.org:0704.0059,2009-06-23,physics:astro-ph,2019-02-08T19:12:29Z
6,oai:arXiv.org:0704.0080,2010-11-11,physics:astro-ph,2019-02-08T19:12:29Z
7,oai:arXiv.org:0704.0094,2007-05-23,physics:astro-ph,2019-02-08T19:12:29Z
8,oai:arXiv.org:0704.0128,2009-11-13,physics:astro-ph,2019-02-08T19:12:29Z
9,oai:arXiv.org:0704.0133,2009-11-13,physics:astro-ph,2019-02-08T19:12:29Z


We need to truncate and, if necessary, alter each identifier to match the actual filename as seen in S3.

There are two types of identifiers:
- `oai:arXiv.org:0704.009` — The filename will be `0704.009`, so we take everything after the last colon. 
- `oai:arXiv.org:quant-ph/0703214` — The filename will be `quant-ph0703214`, so we take everything after the last colon, then remove the slash.

In [230]:
filenames = identifiers_df['identifier'].apply(lambda x: x[x.rfind(':') + 1:].replace('/', ''))
identifiers_df['filename'] = filenames
identifiers_df

,identifier,date_uploaded,setSpec,date_retrieved,filename
0,oai:arXiv.org:0704.0009,2010-03-18,physics:astro-ph,2019-02-08T19:12:29Z,0704.0009
1,oai:arXiv.org:0704.0017,2009-06-23,physics:astro-ph,2019-02-08T19:12:29Z,0704.0017
2,oai:arXiv.org:0704.0023,2009-06-23,physics:astro-ph,2019-02-08T19:12:29Z,0704.0023
3,oai:arXiv.org:0704.0044,2015-05-13,physics:astro-ph,2019-02-08T19:12:29Z,0704.0044
4,oai:arXiv.org:0704.0048,2008-11-26,physics:astro-ph,2019-02-08T19:12:29Z,0704.0048
5,oai:arXiv.org:0704.0059,2009-06-23,physics:astro-ph,2019-02-08T19:12:29Z,0704.0059
6,oai:arXiv.org:0704.0080,2010-11-11,physics:astro-ph,2019-02-08T19:12:29Z,0704.0080
7,oai:arXiv.org:0704.0094,2007-05-23,physics:astro-ph,2019-02-08T19:12:29Z,0704.0094
8,oai:arXiv.org:0704.0128,2009-11-13,physics:astro-ph,2019-02-08T19:12:29Z,0704.0128
9,oai:arXiv.org:0704.0133,2009-11-13,physics:astro-ph,2019-02-08T19:12:29Z,0704.0133


Save data frame to CSV:

In [233]:
identifiers_df.to_csv('arXiv_identifiers.csv', index=False, header=True)

Example reading in CSV without removing leading zeros, especially in filename:

In [235]:
opened_identifiers_df = pd.read_csv('arXiv_identifiers.csv', dtype={'filename': str})
opened_identifiers_df

Now we have all the filenames for all of the astro-ph articles that arXiv specifies in their metadata!